In [3]:
import numpy as np
import sys
import pandas as pd
import copy

# Request generation function

In [4]:
def create_mars_request(reqpath, request,dryrun=True):
    
    """
    Generates a MARS request at location reqpath based on the request dictionary.
    """
    
    req = 'retrieve,\n'
    
    for key,value in request.items():
        req += '{}={},\n'.format(key,value)
        
    if dryrun:
        
        print('saving request below to {}:\n\n{}'.format(reqpath,req))
        return 
        
    with open(reqpath,'w') as f:

        f.write(req)

# Set global variables

In [5]:
## create list of experiment details
exp_list = pd.DataFrame(columns=['id','ptype','fctype','fcdate'])

exps = ['b2nn','b2no','b2nq','b2nr','b2ns','b2nt']
ptype = ['pi','incr']*3
fcdate = ['2022-02-14']*2+['2022-02-10']*2+['2022-02-16']*2

for i in range(6):
    exp_list.loc[i*2] = [exps[i],ptype[i],'cf',fcdate[i]]
    exp_list.loc[i*2+1] = [exps[i],ptype[i],'pf',fcdate[i]]

In [6]:
outdir = '/gf3/predict2/AWH012_LEACH_NASTORM/DATA/'
# outdir = '/home/e/ermis/DATA/'

### Parameter lists

In [7]:
param_longnames = ["10 metre U wind component", "10 metre V wind component", "10 metre wind gust since previous post-processing", "100 metre U wind component", "100 metre V wind component", "2 metre dewpoint temperature", "2 metre temperature", "Albedo", "Average potential temperature in the upper 300m", "Average salinity in the upper 300m", "Averaged total lightning flash density in the last 6 hours", "Boundary layer dissipation", "Boundary layer height", "Charnock", "Convective available potential energy", "Convective inhibition", "Convective precipitation", "Convective rain rate", "Depth of 20C isotherm", "Eastward sea water velocity", "Eastward turbulent surface stress", "Evaporation", "Forecast surface roughness", "Geopotential", "Gravity wave dissipation", "High cloud cover", "Instantaneous 10 metre wind gust", "Instantaneous large-scale surface precipitation fraction", "Instantaneous total lightning flash density", "K index", "Land-sea mask", "Large scale rain rate", "Large-scale precipitation", "Low cloud cover", "Maximum CAPE in the last 6 hours", "Maximum CAPES in the last 6 hours", "Maximum temperature at 2 metres since previous post-processing", "Maximum total precipitation rate since previous post-processing", "Mean sea level pressure", "Medium cloud cover", "Minimum temperature at 2 metres since previous post-processing", "Mixed layer depth", "Northward sea water velocity", "Northward turbulent surface stress", "Potential evaporation", "Precipitation type", "Runoff", "Sea ice area fraction", "Sea surface height", "Sea surface temperature", "Sea water practical salinity", "Sea-ice thickness", "Skin temperature", "Surface latent heat flux", "Surface roughness", "Surface sensible heat flux", "Surface thermal radiation downwards", "Top net solar radiation", "Top net thermal radiation", "Total cloud cover", "Total column vertically-integrated water vapour", "Total column water", "Total precipitation", "Total totals index", "Visibility"]

# param_df = pd.Dataframe(columns = )

# 0.25 deg EU

In [6]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/EU025/sfc/{fctype}/req_files/{exp}_{fcdate}_extravars.req'

param = '172.128' # for ENS
param = '172.128' # for EXP

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
# step_ens = np.arange(6,360,6)
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128/165.128/166.128/246.228/247.228' # for EXP
    
    exp,ptype,fctype,fcdate = S.values
    
    exptype='MED-R/EXP'
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levtype=sfc,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=0.25/0.25,\n'
    req += 'area=E,\n'
    req += 'target="extravars/'+exp+'_'+fcdate+'.nc"\n\n'
    
    ## 6-hrly data
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength] if x%6==0][1:])
    param='228035/228036'
    req += 'retrieve,\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+'\n\n'

    with open(reqpath,'w') as f:

        f.write(req)
                
# sfc - ENS
exptype = 'MED-R/ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2022-02-{:02d}'.format(x) for x in [10,14,16]]:
        
        param = '172.128'

        fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=E,\n'
        req += 'target="extravars/'+exp+'_'+fcdate+'.nc"\n\n'
        
        ## 6-hrly data
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength] if x%6==0][1:])
        param='228035/228036'
        req += 'retrieve,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n\n'
        
        with open(reqpath,'w') as f:

            f.write(req)

In [9]:
# pl - expmkdir extra
basepath = outdir+'{exptype}/{ptype}/EU025/pl/{fctype}/req_files/{exp}_{fcdate}_extravars.req'

param = '129.128/130.128/131/132/133.128/135.128/138.128/155.128/157.128'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    exp,ptype,fctype,fcdate = S.values
    
    exptype='MED-R/EXP'
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levelist=250/500/700/850,\n'
    req += 'levtype=pl,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=0.25/0.25,\n'
    req += 'area=E,\n'
    req += 'target="extravars/'+exp+'_'+fcdate+'.nc"\n\n'

    with open(reqpath,'w') as f:

        f.write(req)
                
# pl - ENS
exptype = 'MED-R/ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2022-02-{:02d}'.format(x) for x in [10,14,16]]:
        
        fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=pl,\n'
        req += 'levelist=250/500/700/850,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=E,\n'
        req += 'target="extravars/'+exp+'_'+fcdate+'.nc"\n\n'
        
        with open(reqpath,'w') as f:

            f.write(req)

# 1 deg global

In [8]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/GLO100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128/130.128/131/132/133.128/135.128/138.128/155.128/157.128'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    exp,ptype,fctype,fcdate = S.values
    
    exptype='MED-R/EXP'
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levtype=sfc,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'area=90/-180/-90/180,\n'
    req += 'grid=1.0/1.0,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        
    with open(reqpath,'w') as f:

            f.write(req)

# sfc - ENS
exptype = 'MED-R/ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2022-02-{:02d}'.format(x) for x in [10,14,16]]:
        
        param = '129.128/130.128/131/132/133.128/135.128/138.128/155.128/157.128'

        fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'area=E,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        ## 6-hrly data
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength] if x%6==0][1:])
        param='228035/228036'
        req += 'retrieve,\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+'\n\n'
        
        with open(reqpath,'w') as f:

            f.write(req)

In [10]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/GLO100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128/130.128/131/132/133.128/135.128/138.128/155.128/157.128'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    exp,ptype,fctype,fcdate = S.values
    
    exptype='MED-R/EXP'
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levelist=250/500/700/850,\n'
    req += 'levtype=pl,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=1.0/1.0,\n'
    req += 'area=90/-180/-90/180,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

    with open(reqpath,'w') as f:

        f.write(req)
                
# pl - ENS
exptype = 'MED-R/ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2022-02-{:02d}'.format(x) for x in [10,14,16]]:
        
        fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=pl,\n'
        req += 'levelist=250/500/700/850,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'area=90/-180/-90/180,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        with open(reqpath,'w') as f:

            f.write(req)